In [1]:
import os

In [2]:
%pwd

'd:\\Artificial Intelligence and Machine Learning\\End2End Projects\\chicken-disease-classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Artificial Intelligence and Machine Learning\\End2End Projects\\chicken-disease-classification'

In [11]:
## Define the entity

from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen = True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path    

In [12]:
from cnnclassifier.constants import *
from cnnclassifier.utils.common import *
from pathlib import Path
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH, 
                 params_filepath = PARAMS_FILE_PATH,
                 secret_filepath = SECRET_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.secret = read_yaml(secret_filepath)

        create_directories_files([self.config.artifacts_root])
    
    def get_callbacks_config(self)-> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        create_directories_files([config.root_dir])
        create_directories_files([config.tensorboard_root_log_dir])
        create_directories_files([os.path.dirname(config.checkpoint_model_filepath)])


        prepare_callbacks_config = PrepareCallbacksConfig(root_dir = Path(config.root_dir), 
                                                          tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir),
                                                          checkpoint_model_filepath = Path(config.checkpoint_model_filepath))
        
        return prepare_callbacks_config
        

In [13]:
## Update components
import tensorflow as tf
import time
import os


In [15]:
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir,
                                          f"tb_logs_at_{timestamp}")
        return tf.keras.callbacks.TensorBoard(log_dir = tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(filepath = self.config.checkpoint_model_filepath, 
                                                  save_best_only = True)
    
    @property
    def _create_reduce_lr_callbacks(self):
        return tf.keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", 
                                                    factor = 0.2, 
                                                    patience = 5, 
                                                    min_lr = 0.001)
    
    @property
    def _create_earlystopping_callbacks(self):
        return tf.keras.callbacks.EarlyStopping(monitor = "val_loss", 
                                                patience = 10)
    
    def get_tb_ckpt_callbacks(self):
        return [self._create_tb_callbacks, 
                 self._create_ckpt_callbacks,
                 self._create_reduce_lr_callbacks, 
                 self._create_earlystopping_callbacks]

In [16]:
## Create Pipeline
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config = prepare_callbacks_config)
    callbacks_lst = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2024-08-30 10:04:17,208: INFO : common : config\config.yaml loaded successfully]
[2024-08-30 10:04:17,224: INFO : common : params\params.yaml loaded successfully]
[2024-08-30 10:04:17,227: INFO : common : secrets\secrets.yaml loaded successfully]
[2024-08-30 10:04:17,230: INFO : common : Parent Directories and Files Successfully Created]
[2024-08-30 10:04:17,232: INFO : common : Parent Directories and Files Successfully Created]
[2024-08-30 10:04:17,233: INFO : common : Parent Directories and Files Successfully Created]
[2024-08-30 10:04:17,235: INFO : common : Parent Directories and Files Successfully Created]
